In [ ]:
from typing import Generic, TypeVar, Dict, List, NamedTuple, Optional
from abc import ABC, abstractmethod
from random import choice
from string import ascii_uppercase


V = TypeVar('V')
D = TypeVar('D')

class Constraint(Generic[V, D], ABC):
    def __init__(self, variables: List[V]) -> None:
        self.variables = variables
        
    @abstractmethod
    def satisfied(self, assignment: Dict[V, D]) -> bool:
        ...

Constraint-satisfaction problems have **variables** V with range of values known as **domains** D and **constraints** which determine if a particular variable's domain selection is valid.

In [ ]:
class CSP(Generic[V, D]):
    def __init__(self, variables: List[V], domains: Dict[V, List[D]]) -> None:
        self.variables: List[V] = variables
        self.domains: Dict[V, List[D]] = domains
        self.constraints: Dict[V, List[Constraint[V, D]]] = {}
            
        for v in self.variables:
            self.constraints[v] = []
            if v not in domains:
                raise LookupError(f'Variable {v} does not have a domain.')
                
    def add_constraint(self, constraint: Constraint[V, D]) -> None:
        for v in constraint.variables:
            if v not in self.variables:
                raise LookupError(f'Variable {v} is in constraint but not in CSP.')
            self.constraints[v].append(constraint)
            
    def consistent(self, variable: V, assignment: Dict[V, D]) -> bool:
        """Check if the value assignment is consistent."""
        for c in self.constraints[variable]:
            if not c.satisfied(assignment):
                return False
        return True
        
    def backtracking_search(self, assignment: Dict[V, D]=None) -> Optional[Dict[V, D]]:
        if not assignment:
            assignment = {}
        
        # assignment is complete if every variable is assigned (base case)
        if len(assignment) == len(self.variables):
            return assignment
        
        # get all the variables in the CSP but not in the assignment
        unassigned: List[V] = [v for v in self.variables if v not in assignment]
            
        # get every possible domain value of the first non-assigned variable
        first: V = unassigned[0]
        for value in self.domains[first]:
            local_assignment = assignment.copy()
            local_assignment[first] = value
            
            # if we're still consistent, we recurse (continue)
            if self.consistent(first, local_assignment):
                result: Optional[Dict[V, D]] = self.backtracking_search(local_assignment)
                    
                # if we did not find the result, we will end up backtracking
                if result is not None:
                    return result
                
        return

# Australia map coloring
We make a class which defines a constraint on how to color a map of Australia and have neighbouring states with different colors.

In [ ]:
class MapColoringConstraint(Constraint[str, str]):
    def __init__(self, place1: str, place2: str) -> None:
        super().__init__([place1, place2])
        self.place1: str = place1
        self.place2: str = place2
            
    def satisfied(self, assignment: Dict[str, str]) -> bool:
        if self.place1 not in assignment or self.place2 not in assignment:
            return True
        
        return assignment[self.place1] != assignment[self.place2]

In [ ]:
variables: List[str] = [
    'Western Australia',
    'Northern Territory',
    'South Australia',
    'Queensland',
    'New South Wales',
    'Victoria',
    'Tasmania'
]

domains: Dict[str, List[str]] = {}
    
for v in variables:
    domains[v] = ['red', 'green', 'blue']

csp: CSP[str, str] = CSP(variables, domains)
csp.add_constraint(MapColoringConstraint('Western Australia', 'Northern Territory'))
csp.add_constraint(MapColoringConstraint('Western Australia', 'South Australia'))
csp.add_constraint(MapColoringConstraint('South Australia', 'Northern Territory'))
csp.add_constraint(MapColoringConstraint('Queensland', 'Northern Territory'))
csp.add_constraint(MapColoringConstraint('Queensland', 'New South Wales'))
csp.add_constraint(MapColoringConstraint('Queensland', 'South Australia'))
csp.add_constraint(MapColoringConstraint('South Australia', 'New South Wales'))
csp.add_constraint(MapColoringConstraint('South Australia', 'Queensland'))
csp.add_constraint(MapColoringConstraint('South Australia', 'Victoria'))
csp.add_constraint(MapColoringConstraint('Victoria', 'New South Wales'))
csp.add_constraint(MapColoringConstraint('Victoria', 'Tasmania'))

In [ ]:
solution: Optional[Dict[str, str]] = csp.backtracking_search()
if solution is None:
    print('No solution found.')
else:
    for k, v in solution.items():
        print(f'{k}: {v}')

# 8 Queens problem
How can we place 8 queens on the checkboard so that they are not attacking each other?

In [ ]:
columns: List[int] = list(range(1, 9))
rows: Dict[int, List[int]] = {}
for c in columns:
    rows[c] = list(range(1, 9))
csp: CSP[int, int] = CSP(columns, rows)
    
class QueenConstraint(Constraint[int, int]):
    def __init__(self, columns: List[int]) -> None:
        super().__init__(columns)
        self.columns: List[int] = columns
        
    def satisfied(self, assignment: Dict[int, int]) -> bool:
        for q1c, q1r in assignment.items():
            for q2c in range(q1c + 1, len(self.columns) + 1):
                if q2c in assignment:
                    q2r: int = assignment[q2c]
                    if q1r == q2r:  # same row
                        return False
                    if abs(q1c - q2c) == abs(q1r - q2r):  # same diagonal
                        return False
        return True

In [ ]:
csp.add_constraint(QueenConstraint(columns))
solution: Optional[Dict[int, int]] = csp.backtracking_search()
solution

# Word search

In [ ]:
Grid = List[List[str]]

class GridLocation(NamedTuple):
    row: int
    column: int
        
def generate_grid(rows: int, columns: int) -> Grid:
    return [[choice(ascii_uppercase) for c in range(columns)] for r in range(rows)]

def display_grid(grid: Grid) -> None:
    for row in grid:
        print(''.join(row))
        
def generate_domain(word: str, grid: Grid) -> List[List[GridLocation]]:
    domain: List[List[GridLocation]] = []
    height: int = len(grid)
    width: int = len(grid[0])
    length: int = len(word)
    for row in range(height):
        for column in range(width):
            rows: range = range(row, row + length + 1)
            columns: range = range(column, column + length + 1)
            if column + length <= width:
                # left to right
                domain.append([GridLocation(row, c) for c in columns])
                # diagonals towards bottom right
                if row + length <= height:
                    domain.append([GridLocation(r, column + (r - row)) for r in rows])
            if row + length <= height:
                # top to bottom
                domain.append([GridLocation(r, column) for r in rows])
                # diagonal towards bottom left
                if column - length >= 0:
                    domain.append([GridLocation(r, c - (r - row)) for r in rows])
    return domain

In [ ]:
[GridLocation(r, c) for r in range(3) for c in range(3)]

In [ ]:
class WordSearchConstraint(Constraint[str, List[GridLocation]]):
    def __init__(self, words: List[str]) -> None:
        super().__init__(words)
        self.words: List[str] = words
            
    def satisfied(self, assignment: Dict[str, List[GridLocation]]) -> bool:
        # if there are any duplicate grid locations, then there is an overlap
        all_locations = [locs for values in assignment.values() for locs in values]
        return len(set(all_locations)) == len(all_locations)

In [ ]:
grid: Grid = generate_grid(9, 9)
words: List[str] = ['MATTHEW', 'JOE', 'MARY', 'SARAH', 'SALLY']
locations: Dict[str, List[List[GridLocation]]] = {}
for word in words:
    locations[word] = generate_domain(word, grid)
csp: CSP[str, List[GridLocation]] = CSP(words, locations)
csp.add_constraint(WordSearchConstraint(words))
# solution: Optional[Dict[str, List[GridLocation]]] = csp.backtracking_search()
# if solution is None:
#     print('No solution was found.')
# else:
#     for word, locations in solution.items():
#         # reverse randomly half the time
#         if choice([True, False]):
#             locations.reverse()
#         for index, letter in enumerate(word):
#             grid[locations[index].row][locations[index].column] = letter
#     display_grid(grid)

# Send More Money

In [ ]:
class SendMoreMoneyConstraint(Constraint[str, int]):
    def __init__(self, letters: List[str]) -> None:
        super().__init__(letters)
        self.letters: List[str] = letters
            
    def satisfied(self, assignment: Dict[str, int]) -> bool:
        # if there are some duplicates values then there is no solution
        if len(set(assignment.values())) < len(assignment):
            return False
        
        # if all variables have been assigned, check it adds correctly
        if len(assignment) == len(self.letters):
            s: int = assignment['S']
            e: int = assignment['E']
            n: int = assignment['N']
            d: int = assignment['D']
            m: int = assignment['M']
            o: int = assignment['O']
            r: int = assignment['R']
            y: int = assignment['Y']
            send = 1000 * s + 100 * e + 10 * n + d
            more = 1000 * m + 100 * o + 10 * r + e
            money = 10000 * m + 1000 * o + 100 * n + 10 * e + y
            return send + more == money
        
        # no conflict
        return True   

In [ ]:
letters: List[str] = ['S', 'E', 'N', 'D', 'M', 'O', 'R', 'Y']
possible_digits: Dict[str, List[int]] = {}
for letter in letters:
    possible_digits[letter] = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
possible_digits['M'] = [1]  # we won't get answers starting with 0
csp: CSP[str, int] = CSP(letters, possible_digits)
csp.add_constraint(SendMoreMoneyConstraint(letters))
solution: Optional[Dict[str, int]] = csp.backtracking_search()
if not solution:
    print('No solution was found.')
else:
    print(solution)